In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2


['https://www.1aauto.com/search?q=brake+calipers',
 'https://www.1aauto.com/weatherstripping/c/12']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('page', '')
dict_param['sort'] = 'name+asc'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'
    
list_url, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'{url}page={i}',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                list_href = html.xpath('//ul[@id="serp-list"]/li/div/div[@class="desktop-product-results"]/descendant::h4/a/@href')
                if list_href:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_url += [f'''https://www.1aauto.com{href.strip()}''' for href in list_href]

    # = = = = = = = = = = = = = = =
    
    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//a[@class="pagination-button pagination-button-next"]'):
        break

resp

1 753
2 753
3 753
4 753
5 753
6 753
7 753
8 753
9 753
10 753
11 753
12 753
10
1 754
2 754
3 754
4 754
5 754
6 754
7 754
8 754
9 754
10 754
11 754
10
1 755
2 755
3 755
4 755
5 755
6 755
7 755
8 755
9 755
10 755
11 755
12 755
13 755
14 755
15 755
16 755
17 755
18 755
19 755
20 755
21 755
22 755
23 755
24 755
25 755
26 755
27 755
1


<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.1aauto.com/honda-civic-insight-fro...
1,2,https://www.1aauto.com/chevrolet-gmc-cadillac-...
2,3,https://www.1aauto.com/honda-accord-acura-cl-r...
3,4,https://www.1aauto.com/honda-accord-acura-cl-r...
4,5,https://www.1aauto.com/ford-mazda-mercury-fron...
5,6,https://www.1aauto.com/ford-mazda-mercury-fron...
6,7,https://www.1aauto.com/ford-front-driver-side-...
7,8,https://www.1aauto.com/dodge-ram-chrysler-fron...
8,9,https://www.1aauto.com/chevrolet-buick-pontiac...
9,10,https://www.1aauto.com/ford-front-passenger-si...


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
